<a href="https://colab.research.google.com/github/iamharkirat/STP-540-Computational-Statistics/blob/main/STP540_Single_Layer_Neural_Network_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV

# Sinle Layer Neural Network Class

In [5]:
class SingleLayerNeuralNetwork:
    def __init__(self, num_inputs=1, num_outputs=1, hidden_units=1, learning_rate=0.1, epochs=100):
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.hidden_units = hidden_units
        self.learning_rate = learning_rate
        self.epochs = epochs
        
        self.weights = np.random.randn(num_inputs, hidden_units)
        self.bias = np.random.randn(hidden_units)
        self.output_weights = np.random.randn(hidden_units, num_outputs)
        self.output_bias = np.random.randn(num_outputs)
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def predict(self, inputs):
        hidden = np.dot(inputs, self.weights) + self.bias
        hidden = self.sigmoid(hidden)
        output = np.dot(hidden, self.output_weights) + self.output_bias
        return self.sigmoid(output)
    
    def sgd(self, inputs, targets):
        for i in range(inputs.shape[0]):
            x = inputs[i]
            y = targets[i]
            
            hidden = np.dot(x, self.weights) + self.bias
            hidden = self.sigmoid(hidden)
            output = np.dot(hidden, self.output_weights) + self.output_bias
            output = self.sigmoid(output)

            error = y - output

            d_output = error * output * (1 - output)
            d_hidden = d_output.dot(self.output_weights.T) * hidden * (1 - hidden)

            d_output_weights = hidden.reshape(-1, 1).dot(d_output.reshape(1, -1))
            d_output_bias = d_output
            d_weights = x.reshape(-1, 1).dot(d_hidden.reshape(1, -1))
            d_bias = d_hidden

            self.weights += self.learning_rate * d_weights
            self.bias += self.learning_rate * d_bias
            self.output_weights += self.learning_rate * d_output_weights
            self.output_bias += self.learning_rate * d_output_bias
    
    def fit(self, X, y):
        for epoch in range(self.epochs):
            self.sgd(X, y)
    
    def get_params(self, deep=True):
        return {"num_inputs": self.num_inputs, 
                "num_outputs": self.num_outputs,
                "hidden_units": self.hidden_units, 
                "learning_rate": self.learning_rate, 
                "epochs": self.epochs}
    
    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

# Simulated Data

In [9]:
# Generate simulated data
np.random.seed(42)
X = np.random.rand(100, 1) * 10
y = np.where(X > 5, 1, 0)

# Run the Model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

def custom_score(model, X, y):
    y_pred = model.predict(X)
    return accuracy_score(y, y_pred.round())


# Define parameter grid
param_grid = {
    'epochs': [50, 100, 150, 300, 500, 1000],
    'learning_rate': [0.1, 0.01, 0.001, 0.0001],
    'hidden_units': [2, 3, 4, 6, 10]
}

# Create and fit the model using Grid Search CV
model = SingleLayerNeuralNetwork(num_inputs=1, num_outputs=1)
grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring=custom_score)
grid_search.fit(X, y)

# Print the best hyperparameters and accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)

# Save hyperparameters and corresponding accuracy in a pandas dataframe
results = pd.DataFrame(grid_search.cv_results_)
results = results[['param_epochs', 'param_learning_rate', 'param_hidden_units', 'mean_test_score']]
results = results.rename(columns={
    'param_epochs': 'Epochs', 
    'param_learning_rate': 'Learning Rate', 
    'param_hidden_units': 'Hidden Units', 
    'mean_test_score': 'Accuracy'
})

# Sort results by accuracy and reset index
results = results.sort_values(by='Accuracy', ascending=False).reset_index(drop=True)
print(results)